<a href="https://colab.research.google.com/github/antmaio/Creactif_Fastai/blob/main/Fastai_Seance6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Données en Tableau

In [ ]:
# Pour traiter les Time Series
!pip install tsai
import tsai

In [ ]:
import torch.nn as nn
import pandas as pd
from fastai.tabular.all import *
torch.cuda.empty_cache()

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/export.pkl'),Path('/root/.fastai/data/adult_sample/adult.csv'),Path('/root/.fastai/data/adult_sample/models')]

Visualisation du tableau (DataFrame) :

In [ ]:
df = pd.read_csv(path/'adult.csv') #lire un fichier excel en DataFrame
df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
df.nunique() #Nombre de classe unique dans chaque colonne

age                  73
workclass             9
fnlwgt            21648
education            16
education-num        16
marital-status        7
occupation           15
relationship          6
race                  5
sex                   2
capital-gain        119
capital-loss         92
hours-per-week       94
native-country       42
salary                2
dtype: int64

NaN = Not a Number ⇒ Donnée non récoltée

## Exercice 1:
Afficher les 10 derniers éléments du tableau

In [ ]:
# Votre code
df.tail(10)

## Création du DataLoader

y_names : ce que l'on veut estimer (sortie du réseau de neurones)

cat_names : les entrées que l'on peut diviser en catégories (discrètes), ex: chat / chien

cont_names : les entrées qui peuvent prendre des valeurs continues, ex: valeur des pixels dans une image

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']

dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
                                   cat_names=cat_names, cont_names=cont_names)

dls = TabularDataLoaders.from_df(df, y_names="salary",
                                   cat_names=cat_names, cont_names=cont_names)


Could not do one pass in your dataloader, there is something wrong in it. Please see the stack trace below:


TypeError: ignored

Erreur car la fonction dataloaders ne comprend pas les données non numériques (idem que chat/chien)

⇒ On utilise la Transform "Categorify":

In [ ]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
                                  cat_names=cat_names, cont_names=cont_names,
                                  procs = Categorify)

test = pd.DataFrame(dls.train_ds[['occupation','education-num']])
test = test.sort_index()
print(test[:5])

   occupation  education-num
0           0           12.0
1           5           14.0
2           0            NaN
4           9            NaN
5           7            9.0


## Exercice 2:
Pourquoi nous n'avons pas les mêmes indices que dans df.head(5) ? Comment les catégories sont-elles définies dans Categorify ?

Composition d'un batch : [entrées catégorielles, entrées continues, sorties]

In [ ]:
print(f'length: {len(dls.one_batch())}')
print(dls.one_batch()[0][:5])
print(dls.one_batch()[1][:5])
print(dls.one_batch()[2][:5])

length: 3
tensor([[ 5, 10,  5, 13,  2,  5],
        [ 7, 12,  7,  9,  2,  5],
        [ 5, 12,  1,  9,  5,  5],
        [ 5,  2,  1,  7,  3,  5],
        [ 5,  2,  5,  9,  4,  5]])
tensor([[2.4000e+01, 1.4000e+05, 1.0000e+01],
        [4.1000e+01, 2.6653e+05, 9.0000e+00],
        [3.3000e+01, 9.3930e+04, 9.0000e+00],
        [5.0000e+01, 1.5098e+05, 9.0000e+00],
        [1.9000e+01, 2.4834e+05, 9.0000e+00]])
tensor([[1],
        [0],
        [0],
        [1],
        [0]], dtype=torch.int8)


Pour gérer les éléments manquants, on utilise la Transform "FillMissing"

Note : les éléments manquants vont être remplacés par la valeur médiane

In [ ]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
                                  cat_names=cat_names, cont_names=cont_names,
                                  procs = [Categorify, FillMissing])
test = pd.DataFrame(dls.train_ds[['occupation','education-num']])
test = test.sort_index()
print(test[:5])

   occupation  education-num
1           5           14.0
2           0           10.0
3          11           15.0
4           9           10.0
7           0            7.0


Pour optimiser le traitement des entrées continues, on utilise la Transform "Normalize"

In [ ]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
                                  cat_names=cat_names, cont_names=cont_names,
                                  procs = [Categorify, FillMissing, Normalize])
test = pd.DataFrame(dls.train_ds[['occupation','education-num']])
test = test.sort_index()
print(test[:5])

   occupation  education-num
0           0       0.752412
1           5       1.535236
2           0      -0.030413
3          11       1.926648
4           9      -0.030413


## Learner


Il y a deux manières de procéder pour pouvoir créer notre Learner : 
1. Soit nous utilisons un Learner relatif aux données tabulaire tabular_learner
2. Soit nous utilisons le learner général de fastai mais cela implique de définir nos paramètres nous-mêmes. Cependant, nous pouvons facilement modifier les paramètres de l'architecture du modèle à utiliser.

In [ ]:
"""Méthode 1)"""
from fastai.tabular.model import TabularModel

dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
                                  cat_names=cat_names, cont_names=cont_names,
                                  procs = [Categorify, FillMissing, Normalize])

tab_learn = tabular_learner(dls, metrics=accuracy)
tab_learn.model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(10, 6)
    (1): Embedding(17, 8)
    (2): Embedding(8, 5)
    (3): Embedding(16, 8)
    (4): Embedding(7, 5)
    (5): Embedding(6, 4)
    (6): Embedding(3, 3)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): Linear(in_features=42, out_features=200, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): LinBnDrop(
      (0): Linear(in_features=200, out_features=100, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=100, out_features=2, bias=True)
    )
  )
)

Analyse du réseau :      
1. Embeddings : Les couches d'embeddings permettent de convertir des indices en vecteur de données. A chaque indice (provenant du Categorify) va être associé un vecteur d'information et nous pouvons donc retrouver ce vecteur à partir de cet indice. En informatique, nous appelons ça une look-up table.
2. Linear : Couche de neurones dont la taille du vecteur d'entrée et celle de celui de sortie est fixe.

In [ ]:
#Embeddings
nombre_de_mots, taille_du_vecteur = 3, 5
embedding = nn.Embedding(nombre_de_mots, taille_du_vecteur)
print('Look Up table size : ', embedding)

idx=torch.LongTensor([0,0,1,1,2,2])
embedding(idx)

Look Up table size :  Embedding(3, 5)


tensor([[-0.5225, -0.9873, -0.6818, -1.8511,  0.4479],
        [-0.5225, -0.9873, -0.6818, -1.8511,  0.4479],
        [-1.9098, -0.7941, -0.6209, -0.7461, -1.3926],
        [-1.9098, -0.7941, -0.6209, -0.7461, -1.3926],
        [-0.4239, -0.2308, -1.3524, -0.7838, -0.7588],
        [-0.4239, -0.2308, -1.3524, -0.7838, -0.7588]],
       grad_fn=<EmbeddingBackward0>)

In [ ]:
idx=torch.LongTensor([3])
embedding(idx)
#throw an error because idx>len(vocabalury)

IndexError: ignored

In [ ]:
tab_learn.model.embeds

ModuleList(
  (0): Embedding(10, 6)
  (1): Embedding(17, 8)
  (2): Embedding(8, 5)
  (3): Embedding(16, 8)
  (4): Embedding(7, 5)
  (5): Embedding(6, 4)
  (6): Embedding(3, 3)
)

Une couche linéaire est une couche qui opère une opération matricielle sur le vecteur d'entrée. Elle est définie par un nombre de neurone d'entrée et un nombre de neurone de sortie
nn(6).svg

In [ ]:
#Linear
#perform the operation linear.weight @ x + b
in_size=5
out_size=2
linear=nn.Linear(in_size, out_size)

print('Shape')
print(linear.weight.shape)
print(linear.bias.shape)

x = torch.rand(in_size)

res = linear.weight @  x + linear.bias

print(f'Shape of Linear operation : {linear.weight.shape}  @  {x.shape}  + {linear.bias.shape} = {res.shape}')

Shape
torch.Size([2, 5])
torch.Size([2])
Shape of Linear operation : torch.Size([2, 5])  @  torch.Size([5])  + torch.Size([2]) = torch.Size([2])


Entrainement de notre modèle

In [ ]:
tab_learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.376964,0.354144,0.833538,00:05


In [ ]:
"""
Méthode 2)
"""

emb_szs = get_emb_sz(dls.train_ds, {}) #infer the size of embeddings from our data
n_cont = len(dls.cont_names) #number of continous var
n_out = get_c(dls) #infer the number of classes from our dataloader

mymodel = TabularModel(emb_szs=emb_szs, n_cont=n_cont, out_sz=n_out, layers=[50,200,100])
learn = Learner(dls, mymodel, metrics=accuracy)

print('Model from Learner : ', learn.model)
print('Model from tabular_learner : ', tab_learn.model)


Model from Learner :  TabularModel(
  (embeds): ModuleList(
    (0): Embedding(10, 6)
    (1): Embedding(17, 8)
    (2): Embedding(8, 5)
    (3): Embedding(16, 8)
    (4): Embedding(7, 5)
    (5): Embedding(6, 4)
    (6): Embedding(3, 3)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): Linear(in_features=42, out_features=50, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): LinBnDrop(
      (0): Linear(in_features=50, out_features=200, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=200, out_features=100, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=T

## Exercice 3

Entrainer le modèle défini par le Learner général

In [ ]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.373244,0.356155,0.828317,00:05


# Création de son propre réseau de neurones

Il est possible de créer sa propre architecture de réseau de neurone grâce à la classe nn.Module de Pytorch qui regroupe des fonctionnalités sur lesquelles se base FastAI pour la définition du réseau

Vous pouvez avoir la liste des couches spécifiques existantes dans la documentation de Pytorch https://pytorch.org/docs/stable/nn.html 

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    #Définition des opérations et couches du réseau
    super(NeuralNetwork, self).__init__()
    self.linear1 = nn.Linear(3, 10)
    self.linear2 = nn.Linear(10, 2)
  def forward(self, x):
    #Application des opérations définies dans __init__ 
    x = self.linear1(x)
    x = self.linear2(x)
    return x

In [ ]:
my_first_model = NeuralNetwork()

x = torch.rand(3)
out = my_first_model(x)

print(out.shape)

torch.Size([2])


## Exercice 4 

Créez un réseau de neurones composées de 4 fonctions linéaires (nn.Linear). Ce réseau permet de de classifier la classe (parmis 2) d'une modalité à partir de 10 données d'entrée. La taille des couches cachées est de 100 neurones.

Testez si votre modèle fonctionne (pas d'erreur invoquée) à partir d'un vecteur random

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    #Définition des opérations et couches du réseau
    super(NeuralNetwork, self).__init__()
    self.linear1 = nn.Linear(10, 100)
    self.linear2 = nn.Linear(100, 100)
    self.linear3 = nn.Linear(100,100)
    self.linear4 = nn.Linear(100, 2)

  def forward(self, x):
    #Application des opérations définies dans __init__ 
    x = self.linear1(x)
    x = self.linear2(x)
    x = self.linear3(x)
    x = self.linear4(x)
    return x

In [ ]:
my_first_model = NeuralNetwork()

x = torch.rand(10)
out = my_first_model(x)

print(out.shape)

torch.Size([2])


# Exercice 5
Entraîner notre réseau à partir de données créées au sein de l'environnement

1) Créer une matrice de nombres aléatoires de 1000 lignes, 10 colonnes

Tips: fonction random.rand de numpy

In [ ]:
data = np.random.rand(1000, 10)
print(data.shape)

(1000, 10)


2) Transformer notre matrice en dataframe et ajouter une colonne "category" contenant des booléens aléatoires

Tips: fonction random.choice de numpy

In [ ]:
"""
1) Transformer np array to pandas dataframe
2) np random choice de numpy
3) Add column "category" avec le résultat en 2)
"""

my_df = pd.DataFrame(data)
my_df.head(5)
category = np.random.choice([False, True], (1000,1))
my_df['category'] = category
my_df.head(5)

,0,1,2,3,4,5,6,7,8,9,category
0,0.982505,0.772514,0.824089,0.355381,0.889302,0.266464,0.939664,0.343277,0.840679,0.827652,True
1,0.457730,0.243915,0.553612,0.502293,0.207059,0.097798,0.670777,0.737854,0.993643,0.034088,True
2,0.630928,0.677194,0.483430,0.834942,0.886588,0.325390,0.152765,0.821895,0.874497,0.939422,False
3,0.518226,0.125055,0.775048,0.515362,0.285680,0.508331,0.272248,0.439079,0.626543,0.557693,False
4,0.437935,0.242396,0.591081,0.354874,0.339674,0.238068,0.609503,0.158134,0.217284,0.336160,True


3) Créer un dls adapté à notre réseau "Linear"

Tips: réflechir à l'utilité des embeddings

In [ ]:
dls = TabularDataLoaders.from_df(my_df,y_names='category',
                                 cat_names=[], cont_names=list(range(10)),
                                 procs = [Categorify, FillMissing, Normalize])

print(f'cat: {dls.one_batch()[0][:5]}')
print(f'cont: {dls.one_batch()[1][:5]}')
print(f'out: {dls.one_batch()[2][:5]}')


cat: tensor([], size=(5, 0), dtype=torch.int64)
cont: tensor([[-1.0465,  1.5476,  1.3797,  0.8348, -1.4881,  0.5417,  0.3818, -0.4408,
          1.1722, -0.6769],
        [-1.0220,  0.7330, -0.2108, -1.3294,  1.1543, -0.8616,  0.6881, -0.0268,
         -0.9580, -0.4136],
        [ 0.9134,  1.1308, -0.4737, -1.2568, -0.4872,  0.2174, -1.0456, -0.3302,
          1.1940,  0.9031],
        [ 1.7072,  1.4334, -1.7158, -1.3953, -0.5287,  1.0360,  1.6524, -0.9232,
         -0.1550, -1.5276],
        [ 0.0353,  0.3591,  1.6525,  0.4858,  1.0320, -1.5267,  0.2247, -1.5892,
          1.3776,  1.4621]])
out: tensor([[0],
        [0],
        [1],
        [0],
        [0]], dtype=torch.int8)


4) Créer un modèle MLP contenant 1 couche cachée de 42 neurones

Tips: l'entrée est composée des cat et des cont

In [ ]:
class my_network(nn.Module):
  def __init__(self):
    #Définition des opérations et couches du réseau
    super(my_network, self).__init__()
    self.linear1 = nn.Linear(10, 42)
    self.linear2 = nn.Linear(42, 2)
  def forward(self, x_cat, x_cont):
    #Application des opérations définies dans __init__ 
    x = x_cont
    x = self.linear1(x)
    x = self.linear2(x)
    return x

5) Entraîner le modèle sur 1 epoch

In [ ]:
# votre code
my_model = my_network()
my_learn = Learner(dls, my_model, metrics=accuracy)
my_learn.fit(1)

ici
la


epoch,train_loss,valid_loss,accuracy,time
0,0.693924,0.698584,0.535000,00:00
